<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con vecinos cercanos (KNN)

Ejemplo de clasificación utilizando vecinos cercanos para la clasificación de clientes y determinar que servicio ofrecerle a cada uno<br>

v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('Telecust1.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Telecust1.csv > Telecust1.csv
    else:
        !wget Telecust1.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Telecust1.csv

### `Telecust1.csv.csv`:
El dataset **`Telecust1.csv.csv`** contiene diferentes tipos de clientes que consumen un servicio de telecomunicación, los cuales deseamos clasificar en 4 categorias.<br> [Dataset source](https://www.kaggle.com/prathamtripathi/customersegmentation)

- **region** --> region, ejemplo 2
- **tenure** --> grado permanencia, ejemplo 40
- **age** --> edad, ejemplo 52
- **income** --> ingresos o sueldo, ejemplo 50 (un número que no representa una moneda real
- **marital** --> si está casado o no
- **address** --> dirección
- **employ** --> empleo
- **retire** --> si está retirado o no
- **genero** --> 0 o 1 (no se sabe cual es cual)


# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("Telecust1.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
df.describe()

In [ ]:
# Exploramos que tan balanceado está el dataset,
# como está repartida las categorias entre los clientes actuales
df['custcat'].value_counts()

In [ ]:
ax = sns.countplot(data=df, x="custcat")

Se puede ver que el dataset está bastante balanceado, no habrá una tendencia marcada

In [ ]:
# Nos quedamos con aquellas columnas que podemos entender su relacion con el objetivo
df_clean = df[['tenure', 'age', 'income', 'marital', 'retire', 'gender', 'custcat']]
df_clean

#### Normalización de los datos

Analizar cual es la distribución de los datos numéricos
- tenure
- age
- income

In [ ]:
sns.displot(data=df_clean, x='age')

In [ ]:
sns.displot(data=df_clean, x='tenure')

In [ ]:
sns.displot(data=df_clean, x='income')

El "ingreso" sigue una distribución normal pero con muchos outliers, es por eso que no utilizaremos el MinMaxScaler sino que se utilizará el StandardScaler a pesar de que "tenure" no siga una distribución normal

In [ ]:
from sklearn.preprocessing import StandardScaler
df_norm = df_clean.copy()
age_scaler = StandardScaler()
tenure_scaler = StandardScaler()
income_scaler = StandardScaler()
df_norm.loc[:, 'age'] = age_scaler.fit_transform(df[['age']])
df_norm.loc[:, 'tenure'] = tenure_scaler.fit_transform(df[['tenure']])
df_norm.loc[:, 'income'] = income_scaler.fit_transform(df[['income']])
df_norm.head()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
X = df_norm.drop('custcat', axis=1).values
y = df_norm['custcat'].values
X.shape

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
from sklearn.model_selection import train_test_split
# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Creamos el modelo base
class RandomBaseModel():
    def __init__(self):
        self.classes_ = [0, 1]
    def fit(self,X, y):
        self.classes_ = np.unique(y)
        return None

    def predict(self,X):
        rand = np.random.randint(0, len(self.classes_), size=X.shape[0])
        rand_clases = [self.classes_[x] for x in rand]
        return np.asarray(rand_clases)

In [ ]:
# Obtener la salida según el modelo base
random_model = RandomBaseModel()
random_model.fit(X_train, y_train)
y_hat_base = random_model.predict(X_test)
random_model.classes_

#### Crear un modelo de clasificación con vecinos cercanos (KNN)
Parámetros
- n_neighbors --> (k) número de vecinos cercanos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

K_MAX = 20
mean_acc = np.zeros((K_MAX))

for i in range(K_MAX):
    # Entrenar el modelo
    clf = KNeighborsClassifier(n_neighbors=(i+1)).fit(X_train,y_train)

    # Prediccion
    y_hat = clf.predict(X_test)   

    # Evaluar el modelo
    mean_acc[i] = accuracy_score(y_test, y_hat)

plt.plot(range(1, K_MAX+1), mean_acc,'darkBlue')
plt.ylabel('Accuracy')
plt.xlabel('K')
plt.tight_layout()
plt.show()

print(f"La mejor exactitud se obtuvo con {mean_acc.max():.2f} con K={mean_acc.argmax()+1}")

In [ ]:
clf = KNeighborsClassifier(n_neighbors=13).fit(X_train,y_train)
y_hat = clf.predict(X_test)   

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat_base, normalize=True)

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat, normalize=True)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_hat)
cmd = ConfusionMatrixDisplay(cm, display_labels=clf.classes_)
cmd.plot(cmap=plt.cm.Blues)
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Supongamos que deseamos ver a que categoría pertenecemos
# dado los siguientes datos
age = 25
tenure = 4
income = df['income'].mean() # ganamos el promedio
marital = 0 # no estamos casados
retire = 0 # no estamos retirados
gender = 1 # solo algun genero

In [ ]:
# El scaler espera como entrada una matriz (filas y columnas)
# Por eso el doble corchete
age_numpy = np.array([[age]])
# Utilizamos float para convertir la matriz que retorna el scaler
# a un número
age_norm = float(age_scaler.transform(age_numpy))
tenure_norm = float(tenure_scaler.transform(np.array([[tenure]])))
income_norm = float(income_scaler.transform(np.array([[income]])))
# El sistema espera como entrada "X" en este caso una sola fila pero varias
# columnas, por eso el reshape(1, -1) donde el "-1" significa "varias"
# (el sistema determina cuantas)
X_prueba = np.array([tenure_norm, age_norm, income_norm, marital, retire, gender]).reshape(1, -1)
print('Shape:', X_prueba.shape)
print('Valores:\n', X_prueba)

In [ ]:
mi_categoria = clf.predict(X_prueba)
mi_categoria

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

En este ejemplo se obtuvo muy poca performance, pero se pudo ver como comparar muchos modelos KNN con distintos "K" y a su vez como ingresar un dato nuevo para predecir una categoría. Se podría probar con otros clasificadores pero el problema radica en la falta de datos